In [70]:
import sys, os, importlib
import rasterio

import pandas as pd
import geopandas as gpd

from rasterio import MemoryFile
from contextlib import contextmanager

sys.path.append("../src")

import GOSTRocks.rasterMisc as rMisc
from GOSTRocks.misc import tPrint

In [73]:
in_admin_file = "/home/public/Data/COUNTRY/ETH/ADMIN/gadm36_2_hoa.shp"
inD = gpd.read_file(in_admin_file)

ntl_folder = "/home/public/Data/GLOBAL/NighttimeLights/LI_HARMONIZED"
ntl_files = [os.path.join(ntl_folder, x) for x in os.listdir(ntl_folder)]

In [105]:
@contextmanager
def create_rasterio_inmemory(src, curData):
    '''Create a rasterio object in memory from a 
    
    :param: src - data dictionary describing the rasterio template i.e. - rasterio.open().profile
    :param: curData - numpy array from which to create rasterio object
    '''
    with MemoryFile() as memFile:
        with memFile.open(**src) as dataset:
            dataset.write(curData)
            del curData
        
        with memFile.open() as dataset:
            yield(dataset)

In [75]:
ntl_file = ntl_files[0]
for ntl_file in ntl_files:
    file_name = "_".join(os.path.basename(ntl_file.replace(".tif", "")).split("_")[-2:])
    inR = rasterio.open(ntl_file)
    rData = inR.read()
    for thresh in [7,20,30]:
        tPrint(f"{file_name}: {thresh}")
        curData = (rData > thresh) * rData
        with create_rasterio_inmemory(inR.profile, curData) as curR:
            res = rMisc.zonalStats(inD, curR)
            xx = pd.DataFrame(res, columns=["SUM","MIN",'MAX','MEAN'])
            inD[f"{file_name}_{thresh}"] = xx['SUM']


10:21:08	2008_calDMSP: 7
10:21:11	2008_calDMSP: 20
10:21:14	2008_calDMSP: 30
10:21:18	2009_calDMSP: 7
10:21:21	2009_calDMSP: 20
10:21:24	2009_calDMSP: 30
10:21:27	2010_calDMSP: 7
10:21:30	2010_calDMSP: 20
10:21:33	2010_calDMSP: 30
10:21:37	2011_calDMSP: 7
10:21:39	2011_calDMSP: 20
10:21:42	2011_calDMSP: 30
10:21:46	2012_calDMSP: 7
10:21:49	2012_calDMSP: 20
10:21:51	2012_calDMSP: 30
10:21:54	2013_calDMSP: 7
10:21:57	2013_calDMSP: 20
10:22:00	2013_calDMSP: 30
10:22:03	2014_simVIIRS: 7
10:22:05	2014_simVIIRS: 20
10:22:08	2014_simVIIRS: 30
10:22:11	2015_simVIIRS: 7
10:22:14	2015_simVIIRS: 20
10:22:16	2015_simVIIRS: 30
10:22:20	2016_simVIIRS: 7
10:22:22	2016_simVIIRS: 20
10:22:25	2016_simVIIRS: 30
10:22:28	2017_simVIIRS: 7
10:22:30	2017_simVIIRS: 20
10:22:33	2017_simVIIRS: 30
10:22:36	2018_simVIIRS: 7
10:22:39	2018_simVIIRS: 20
10:22:41	2018_simVIIRS: 30
10:22:45	1992_calDMSP: 7
10:22:47	1992_calDMSP: 20
10:22:50	1992_calDMSP: 30
10:22:53	1993_calDMSP: 7
10:22:55	1993_calDMSP: 20
10:22:58	1

In [104]:
inD = inD.reindex(sorted(inD.columns), axis=1)
finalD = inD.copy()
for col in finalD.columns:
    try:
        finalD[col] = finalD[col].astype(float)
    except:
        pass

In [103]:
finalD.to_file("/home/wb411133/Code/gostrocks/data/RAW/gadm2_HoA_SoL.geojson", driver="GeoJSON")
pd.DataFrame(finalD.drop(['geometry'], axis=1)).to_csv("/home/wb411133/Code/gostrocks/data/RAW/gadm2_HoA_SoL.csv")